# Programming Assignment #3: Expression Trees

An expression tree is a tree where internal nodes are operators and the leaves are either variables or constants. For example, here is an expression tree that corresponds to the expression `2-3*x`:

    (MINUS 2 (TIMES 3 x))

In this assignment, you will 
1. Write an evaluator for expression trees, e.g., the value of the tree above is `-28` when `x=10`.
2. Perform some simple optimizations, e.g., `(MINUS 2 (TIMES (PLUS 1 2) x))` simplifies to the expression above.
3. Show that the optimizations are correct, i.e., that they return the same values.

> Note: You may know that the Java/C++/C#/whatever compiler you use **optimizes** the generated code. In this assignment you will get a feel for what an optimizing compiler does. Coincidentally, the next section introduces the ACL2 `LET*` syntax, and that turns out to be very similar to a code represerntation called *static single assignment (SSA)* that is often used in compilers for optimization. You can learn more about this if you take the compilers course!


## Introducing LET*

You do **not** have to use `LET*` to do this assignment, but `LET*` is a nice feature that can make it easier to write some of the functions below. Here's how it works.

Suppose you want to write a complicated expression, like

    (+ (* 2 3)
       (* 2 3))

Notice that the same term appears twice? `LET*` is simply a way to create a "temporary variable" that has a value and can them be used freely. To be clear, these are **not** variables in the Java or C++ sense, since they can never be changed. It's really just a way to name a value that appears more than once.

The expression above can be written using `LET*` like this:

    (LET* ((x (* 2 3))
           )
          (+ x x))

Here, the common subexpression `(* 2 3)` is given the name `x`. You can define more than one variable. For example,
consider this expression:

    (+ (* 2 (+ 2 1))
       (* 2 (+ 2 1)))

This can be written using `LET*` as

    (LET* ((y (+ 2 1))
           (x (* 2 y))
           )
          (+ x x))

It's up to you whether to use `LET*` or just ignore it. I used it in some of the definitions and not in others, just to convince myself that you could do either.

## TODO-1 (10 pts)

Your first task is to define a datatype for expression trees. The leaves should be either rationals or symbols (like x and y). The internal nodes are operations, which mostly take two arguments. The operations are
* `PLUS`
* `MINUS`
* `TIMES`
* `DIVIDE`
* `UMINUS` (unary minus, so it takes only **one** argument)

In [77]:
(defsnapshot from-the-top)

(defsnapshot todo-1)

(defdata tree (oneof (list (oneof 'PLUS 'MINUS 'TIMES 'DIVIDE) tree tree) (list 'UMINUS tree) :rational :symbol))



ACL2S !>>(DEFSNAPSHOT FROM-THE-TOP)
          20:x(DEFMACRO DEFSNAPSHOT (LABEL) ...)

Summary
Form:  ( DEFLABEL FROM-THE-TOP ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 FROM-THE-TOP
ACL2S !>>(DEFSNAPSHOT TODO-1)

Summary
Form:  ( DEFLABEL TODO-1 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-1
ACL2S !>>(DEFDATA TREE
                  (ONEOF (LIST (ONEOF 'PLUS 'MINUS 'TIMES 'DIVIDE)
                               TREE TREE)
                         (LIST 'UMINUS TREE)
                         RATIONAL SYMBOL))
 Predicate events...
Form:  ( DEFUN TREEP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
 (TREEP ACL2::V1) <= body -- not complete. 
Reasons: 
("The formula fails to fit any of the forms for acceptable :TAU-SYSTEM rules."
 "Nesting i.e. (P (f ... (g x1 ...) ...) not allowed in conclusio

## TODO-2 (10 pts)

Soon we will be evaluating expressions like `(MINUS 2 (TIMES 3 x))`, but before we get there we have to solve the problem of variables. What we need is a data structure that maps variable names to their current values. We will implement with a list of name/value pairs, e.g.,

    '((x 10)
      (y 3)
      (z -4)
      (abc 50))

Given that list, the value of the expression above is `-28`.

Start by defining the data type `env` that maps symbols to rational numbers as above.

> Note: `env` stands for "environment", though this data structure goes by various names, such as "dictionary", "map", "memory", "associative array", "content-addressible memory", "hashtable", and even "object".

In [78]:
(defsnapshot todo-2)

(defdata env (listof (list :symbol :rational)))



ACL2S !>>(DEFSNAPSHOT TODO-2)

Summary
Form:  ( DEFLABEL TODO-2 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-2
ACL2S !>>(DEFDATA ENV (LISTOF (LIST SYMBOL RATIONAL)))
 Predicate events...
Form:  ( DEFUN ENVP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Alistof theory events...
Form:  ( DEFTHM ENVP-IMPLIES-ALISTP ...)
Form:  ( DEFTHM ENVP-IMPLIES-TLP ...)
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
 (ENVP ACL2::V1) <= body -- not complete. 
Reasons: 
("Nesting i.e. (P (f ... (g x1 ...) ...) not allowed in conclusion of signature rule")

 (ENVP ACL2::V1) => body -- not complete. 
Reasons: 
("Nesting i.e. (P (f ... (g x1 ...) ...) not allowed in conclusion of signature rule")

Form:  ( DEFTHM DEF=>ENV ...)
Form:  ( DEFTHM ENV=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-ENV-BUILTIN ...)
Fo

## TODO-3 (10 pts)

Now, define the function `(lookup var environment)` that returns the current value of the variable `var` in the given `environment`. If the variable is not given a value in the environment, return a reasonable default, e.g., `0`.

In [79]:
(defsnapshot todo-3)
(definec lookup (var :symbol environment :env) :rational
    (if (endp environment)
        0
        (if (equal (first (first environment)) var)
            (second (first environment))
            (lookup var (rest environment))
        )
    )
)

(check-expect (lookup 'x (list (list 'y 90) (list 'c 60) (list 'z 5) (list 'x 3) (list 'p 38))) 3)
(check-expect (lookup 'y (list (list 'y 90) (list 'c 60) (list 'z 5) (list 'x 3) (list 'p 38))) 90)
(check-expect (lookup 'c (list (list 'y 90) (list 'c 60) (list 'z 5) (list 'x 3) (list 'p 38))) 60)
(check-expect (lookup 'z (list (list 'y 90) (list 'c 60) (list 'z 5) (list 'x 3) (list 'p 38))) 5)
(check-expect (lookup 'p (list (list 'y 90) (list 'c 60) (list 'z 5) (list 'x 3) (list 'p 38))) 38)



ACL2S !>>(DEFSNAPSHOT TODO-3)

Summary
Form:  ( DEFLABEL TODO-3 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-3
ACL2S !>>(DEFINEC LOOKUP (VAR SYMBOL ENVIRONMENT ENV)
                  RATIONAL
                  (IF (ENDP ENVIRONMENT)
                      0
                      (IF (EQUAL (FIRST (FIRST ENVIRONMENT)) VAR)
                          (SECOND (FIRST ENVIRONMENT))
                          (LOOKUP VAR (REST ENVIRONMENT)))))

Form:  ( TEST-DEFINITION LOOKUP ... )
Form:  ( TEST-BODY-CONTRACTS LOOKUP... ) 
Form:  ( TEST-FUNCTION-CONTRACT LOOKUP ...) 
Testing: Done 
Elapsed Run Time: 1.22 seconds
Form:  ( ADMIT-DEFINITION LOOKUP ... )
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
Form:  ( PROVE-FUNCTION-CONTRACT LOOKUP ... )
Time:  0.14 seconds (prove: 0.08, print: 0.00, other: 0.06)
Form:  ( PROVE-BODY-CONTRACTS LOOKUP ... )
Time:  0.09 seconds (prove: 0.03, print: 0.00, other: 0.06)
Elapsed Run Time: 0.30 seconds
Function Name

## TODO-4 (10 points)

Now is the time! Write the function `(expr-eval tree environment)` that evaluates the given expression using the variable values in the given environment.

> Note: You cannot divide by zero! If the expression performs a division by zero, just treat the result of that division as `0`, for example. (It should probably return `NIL` in that case, but returning `0` instead saves a bit of work.

> Hint: Think about what using a postorder traversal.

In [80]:
(defsnapshot todo-4)

(definec expr-eval (tree :tree environment :env) :rational
    (if (rationalp tree)
        tree
        (if (symbolp tree)
            (lookup tree environment)
            (if (equal (first tree) 'PLUS)
                (+ (expr-eval (second tree) environment) (expr-eval (third tree) environment))
                (if (equal (first tree) 'MINUS)
                    (- (expr-eval (second tree) environment) (expr-eval (third tree) environment))
                    (if (equal (first tree) 'UMINUS)
                        (- 0 (expr-eval (second tree) environment))
                        (if (equal (first tree) 'TIMES)
                            (* (expr-eval (second tree) environment) (expr-eval (third tree) environment))
                            (if (equal (first tree) 'DIVIDE)
                                (if (equal (expr-eval (third tree) environment) 0)
                                    0
                                    (/ (expr-eval (second tree) environment) (expr-eval (third tree) environment))
                                )
                                0
                            )
                        )
                    )
                )   
            )   
        )   
    )
)

(check-expect (expr-eval  (list 'MINUS 2 (list 'TIMES (list 'PLUS 1 2) 'x)) (list (list 'x 10))) -28)
(check-expect (expr-eval  (list 'MINUS 2 (list 'TIMES 3 'x)) (list (list 'x 10))) -28)
(check-expect (expr-eval  (list 'PLUS 2 (list 'TIMES (list 'TIMES -6 15) 'y)) (list (list 'y 4))) -358)
(check-expect (expr-eval  (list 'DIVIDE (list 'MINUS (list 'UMINUS 6) 'x) 0) (list (list 'y 89))) 0)
(check-expect (expr-eval  (list 'PLUS 2 (list 'DIVIDE (list 'TIMES 88 2) 'x)) (list (list 'x 0))) 2)


ACL2S !>>(DEFSNAPSHOT TODO-4)

Summary
Form:  ( DEFLABEL TODO-4 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-4
ACL2S !>>(DEFINEC
          EXPR-EVAL (TREE TREE ENVIRONMENT ENV)
          RATIONAL
          (IF
           (RATIONALP TREE)
           TREE
           (IF
            (SYMBOLP TREE)
            (LOOKUP TREE ENVIRONMENT)
            (IF
              (EQUAL (FIRST TREE) 'PLUS)
              (+ (EXPR-EVAL (SECOND TREE) ENVIRONMENT)
                 (EXPR-EVAL (THIRD TREE) ENVIRONMENT))
              (IF (EQUAL (FIRST TREE) 'MINUS)
                  (- (EXPR-EVAL (SECOND TREE) ENVIRONMENT)
                     (EXPR-EVAL (THIRD TREE) ENVIRONMENT))
                  (IF (EQUAL (FIRST TREE) 'UMINUS)
                      (- 0 (EXPR-EVAL (SECOND TREE) ENVIRONMENT))
                      (IF (EQUAL (FIRST TREE) 'TIMES)
                          (* (EXPR-EVAL (SECOND TREE) ENVIRONMENT)
                             (EXPR-EVAL (THIRD TREE) ENVIRON

## Extra Credit 1 (10 points)
   
The function `expr-eval` in TODO-4 returns 0 for the expression `(/ 1 0)`. That's OK for convenience, but it's not really right, is it? Let's write a new function `(expr-eval-nan tree environment)` that handles this case properly. The function should return either a rational or the special constant `NaN` ("Not a Number"). When the function sees a term that is trying to divide by 0, it should return `NaN`. Moreover, when it performs any arithmetic operation on a `NaN` value, it should also return `NaN`. For instance

    (PLUS (TIMES 4 x) (DIVIDE 2 0))

should evaluate to `NaN`, regardless of the value of `x`.

In [81]:
(defsnapshot ec-1)

;(or (rationalp tree) (equal tree 'NaN))


(defdata return (oneof :rational 'NaN))
(definec expr-eval-nan (tree :tree environment :env) :return
    (if (or (rationalp tree) (equal tree 'NaN))
        tree
        (if (symbolp tree)
            (lookup tree environment)
            (if (or (equal (expr-eval-nan (second tree) environment) 'NaN) (and (equal (expr-eval-nan (third tree) environment) 'NaN) (not (equal (first tree) 'UMINUS))))
                'NaN         
                (if (equal (first tree) 'PLUS)
                    (+ (expr-eval-nan (second tree) environment) (expr-eval-nan (third tree) environment))
                    (if (equal (first tree) 'MINUS)
                        (- (expr-eval-nan (second tree) environment) (expr-eval-nan (third tree) environment))
                        (if (equal (first tree) 'UMINUS)
                            (- 0 (expr-eval-nan (second tree) environment))
                            (if (equal (first tree) 'TIMES)
                                (* (expr-eval-nan (second tree) environment) (expr-eval-nan (third tree) environment))
                                (if (equal (first tree) 'DIVIDE)
                                    (if (equal (expr-eval-nan (third tree) environment) 0)
                                        'NaN
                                        (/ (expr-eval-nan (second tree) environment) (expr-eval-nan (third tree) environment))
                                    )
                                    'NaN
                                )
                            )
                        )
                    )   
                )  
            )
        )   
    )
)

(check-expect (expr-eval-nan  (list 'DIVIDE (list 'MINUS (list 'UMINUS 6) 4) 'x) (list (list 'y 89))) 'NaN)
(check-expect (expr-eval-nan (list 'PLUS (list 'TIMES 4 'x) (list 'DIVIDE 2 0)) (list (list 'x 4))) 'NaN)
(check-expect (expr-eval-nan  (list 'DIVIDE 2 (list 'DIVIDE (list 'PLUS 1 2) 'x)) (list (list 'x 10))) 20/3)
(check-expect (expr-eval-nan  (list 'MINUS 2 (list 'TIMES 3 'x)) (list (list 'x 10))) -28)
(check-expect (expr-eval-nan (list 'DIVIDE 0 0) (list (list 'x 4))) 'NaN)


ACL2S !>>(DEFSNAPSHOT EC-1)

Summary
Form:  ( DEFLABEL EC-1 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 EC-1
ACL2S !>>(DEFDATA RETURN (ONEOF RATIONAL 'NAN))
 Predicate events...
Form:  ( DEFUN RETURNP ...)
Form:  ( IN-THEORY (DISABLE* ...))
Form:  ( IN-THEORY (ENABLE ...))
Form:  ( TABLE ACL2::RULESET-TABLE ...)
Form:  ( MAKE-EVENT (LET* ...))
 Tau characterization events...
Defdata/Note: RETURNP relatively complete for Tau.
Form:  ( DEFTHM ACL2::DEF=>RETURN ...)
Form:  ( DEFTHM ACL2::RETURN=>DEF ...)
 Enumerator events...
Form:  ( DEFUN NTH-RETURN-BUILTIN ...)
Form:  ( DEFUN NTH-RETURN/ACC-BUILTIN ...)
Form:  ( PROGN (SET-BOGUS-DEFUN-HINTS-OK T) ...)
Form:  ( ENCAPSULATE NIL (LOGIC) ...)
Time:  0.05 seconds (prove: 0.01, print: 0.00, other: 0.04)
 Registering type...
Form:  ( DEFUN NTH-RETURN ...)
Form:  ( ENCAPSULATE (((NTH-RETURN * ...) ...) ...) ...)
Form:  ( DEFUN NTH-RETURN/ACC ...)
Form:  ( ENCAPSULATE (((NTH-RETURN/ACC * ...) ...) ...) ...)
Form

## TODO-5 (10 points)

We'll start the process of optimizing expressions with something simple. Suppose that you see an expressions like `(PLUS x 0)` where `x` is **any** expression (not just the letter `x`). Then clearly, that expression can be replaced with `x`, which should be faster to execute. That's what we mean by optimization. Here's a concrete example:

    (TIMES (PLUS (MINUS x y) 0) 3) ==> (TIMES (MINUS x y) 3)

Notice that the subexpression that was optimized is not necessarily at the top-level.

Write the function `(remove-plus-0 tree)` that removes **all** occurrences of `(PLUS x 0)` and `(PLUS 0 y)` from the given expression tree.

> Hint: Think about what using a postorder traversal again.

In [82]:
(defsnapshot todo-5)

(definec remove-plus-0 (tree :tree) :tree
    (if (or (rationalp tree) (symbolp tree))
        tree
        (if (equal (first tree) 'UMINUS)
            (cons (first tree) (list (remove-plus-0 (second tree))))
            (if (equal (first tree) 'PLUS)
                (if (equal (second tree) 0)
                    (remove-plus-0 (third tree))
                    (if (equal (third tree) 0)
                        (remove-plus-0 (second tree))
                        (append (cons (first tree) (list (remove-plus-0 (second tree)))) (list (remove-plus-0 (third tree))))
                    )
                )  
                (append (cons (first tree) (list (remove-plus-0 (second tree)))) (list (remove-plus-0 (third tree))))
            )     
        ) 
    )
)


(check-expect (remove-plus-0 (list 'TIMES (list 'PLUS (list 'MINUS 'x 'y) 0) 3)) (list 'TIMES (list 'MINUS 'x 'y) 3))
(check-expect (remove-plus-0 (list 'PLUS 'A 'A)) (list 'PLUS 'A 'A))
(check-expect (remove-plus-0 (list 'PLUS 'x 0)) 'x)
(check-expect (remove-plus-0 (list 'PLUS 0 'x)) 'x)
(check-expect (remove-plus-0 (list 'TIMES (list 'PLUS (list 'PLUS 0 'y) 0) 3)) (list 'TIMES 'y 3))


ACL2S !>>(DEFSNAPSHOT TODO-5)

Summary
Form:  ( DEFLABEL TODO-5 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-5
ACL2S !>>(DEFINEC
          REMOVE-PLUS-0 (TREE TREE)
          TREE
          (IF
           (OR (RATIONALP TREE) (SYMBOLP TREE))
           TREE
           (IF
              (EQUAL (FIRST TREE) 'UMINUS)
              (CONS (FIRST TREE)
                    (LIST (REMOVE-PLUS-0 (SECOND TREE))))
              (IF (EQUAL (FIRST TREE) 'PLUS)
                  (IF (EQUAL (SECOND TREE) 0)
                      (REMOVE-PLUS-0 (THIRD TREE))
                      (IF (EQUAL (THIRD TREE) 0)
                          (REMOVE-PLUS-0 (SECOND TREE))
                          (APPEND (CONS (FIRST TREE)
                                        (LIST (REMOVE-PLUS-0 (SECOND TREE))))
                                  (LIST (REMOVE-PLUS-0 (THIRD TREE))))))
                  (APPEND (CONS (FIRST TREE)
                                (LIST (REMOVE-PLUS-0 (SECOND

## TODO-6 (10 points)

If your code for `remove-plus-0` is correct, then the value of the original expression should be the same as the value of the optimized expression, for any given environment. Use `test?` to confirm this property.

> Hint: Don't forget to have a type hypothesis for each variable.

In [83]:
(defsnapshot todo-6)

(test? (implies (and (treep m) (envp e)) (equal (expr-eval-nan m e) (expr-eval-nan (remove-plus-0 m) e))))


ACL2S !>>(DEFSNAPSHOT TODO-6)

Summary
Form:  ( DEFLABEL TODO-6 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-6
ACL2S !>>(TEST? (IMPLIES (AND (TREEP M) (ENVP E))
                         (EQUAL (EXPR-EVAL-NAN M E)
                                (EXPR-EVAL-NAN (REMOVE-PLUS-0 M) E))))

**Summary of Cgen/testing**
We tested 3000 examples across 2 subgoals, of which 2998 (2998 unique)
satisfied the hypotheses, and found 0 counterexamples and 2998 witnesses.

Cases in which the conjecture is true include:
 [found in : "top"]
 -- ((E '((A -1) (BA 0))) (M '(UMINUS A)))
 -- ((E NIL) (M 131/1759))
 -- ((E '((ABBAAAA -1))) (M '|5|))

Test? succeeded. No counterexamples were found.

## TODO-7 (10 points)

The test in TODO-6 should not find any counterexample. If it does find one, either there's a bug in your code (so fix it), or you did not write down the property correctly (so fix that). But if you look at the output, you will probably find that the test is not very convincing. E.g., the patter `(PLUS ... 0)` is rare enough that it never comes up in the random tests.

Let's make sure the code really is working. Use `thm` to **prove** the property fro TODO-6.

In [84]:
(defsnapshot todo-7)

(thm (implies (and (treep m) (envp e)) (equal (expr-eval-nan m e) (expr-eval-nan (remove-plus-0 m) e))))


ACL2S !>>(DEFSNAPSHOT TODO-7)

Summary
Form:  ( DEFLABEL TODO-7 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-7
ACL2S !>>(THM (IMPLIES (AND (TREEP M) (ENVP E))
                       (EQUAL (EXPR-EVAL-NAN M E)
                              (EXPR-EVAL-NAN (REMOVE-PLUS-0 M) E))))

ACL2 Warning [Invariant-risk] in ACL2::TEST-CHECKPOINT:  Invariant-
risk has been detected for a call of function ACL2::TEST-CHECKPOINT
(as possibly leading to an ill-guarded call of CGEN::UI); see :DOC
invariant-risk.


ACL2 Warning [Invariant-risk] in ACL2::TEST-CHECKPOINT:  Invariant-
risk has been detected for a call of function ACL2::TEST-CHECKPOINT
(as possibly leading to an ill-guarded call of CGEN::UI); see :DOC
invariant-risk.


Name the formula above *1.

Perhaps we can prove *1 by induction.  Six induction schemes are suggested
by this conjecture.  Subsumption reduces that number to four.  These
merge into two derived induction schemes.  However, one of these is
fl

## TODO-8 (10 points)

Here is another opportunity for optimization. Consider the expression

    (PLUS (TIMES 2 3) x)

where `x` is **any** expression (not just the variable `x`). This is clearly the same as

    (PLUS 6 x)

Write a function `(fold-constants tree)` that removes all constant subexpressions from the given expression tree. A constant subexpression is one whose arguments are constant terms. For example,

    (PLUS (TIMES 2 (MINUS 6 3)) (UMINUS 5)) ==> 1

That's an extreme example where the entire expression was really a constant.
    

In [85]:
(defsnapshot todo-8)

(definec fold-constants (tree :tree) :tree
    (if (rationalp tree)
        tree
        (if (symbolp tree)
            tree  
            (if (or (equal (fold-constants (second tree)) 'NaN) (and (equal (fold-constants (third tree)) 'NaN) (not (equal (first tree) 'UMINUS))))
                'NaN   
                (if (and (equal (first tree) 'UMINUS) (not (rationalp (fold-constants (second tree)))))
                    (list 'UMINUS (second tree))
                         (if (equal (first tree) 'UMINUS)
                            (- 0 (fold-constants (second tree)))
                             (if (or (not (rationalp (fold-constants (second tree)))) (not (rationalp (fold-constants (third tree)))))
                                 (list (first tree) (fold-constants (second tree)) (fold-constants (third tree)))
                                 (if (not (rationalp (fold-constants (second tree))))
                                     (list (first tree) (second tree) (fold-constants (third tree)))
                                     (if (not (rationalp (fold-constants (third tree))))
                                        (list (first tree) (fold-constants (second tree)) (third tree))                       
                                        (if (equal (first tree) 'PLUS)
                                            (+ (fold-constants (second tree)) (fold-constants (third tree)))
                                            (if (equal (first tree) 'MINUS)
                                                (- (fold-constants (second tree)) (fold-constants (third tree)))
                                                (if (equal (first tree) 'TIMES)
                                                    (* (fold-constants (second tree)) (fold-constants (third tree)))
                                                    (if (equal (first tree) 'DIVIDE)
                                                        (if (equal (fold-constants (third tree)) 0)
                                                            'NaN
                                                            (/ (fold-constants (second tree)) (fold-constants (third tree)))
                                                        )
                                                        'NaN
                                                    )
                                                )
                                            )
                                        )
                                    )
                                )
                            )
                        )
                    )   
                )  
            )
        )   
    )



(check-expect (fold-constants (list 'PLUS (list 'TIMES 2 3) 'x)) (list 'PLUS 6 'x))
(check-expect (fold-constants (list 'PLUS (list 'MINUS 2 7) (list 'PLUS (list 'DIVIDE 1 7) 'x))) (list 'PLUS -5 (list 'PLUS 1/7 'x)))
(check-expect (fold-constants (list 'PLUS (list 'TIMES 2 3) (list 'TIMES (list 'DIVIDE 7 4) 'x))) (list 'PLUS 6 (list 'TIMES 7/4 'x)))
(check-expect (fold-constants (list 'PLUS (list 'TIMES 2 (list 'MINUS 6 3)) (list 'UMINUS 5))) 1)
(check-expect (fold-constants (list 'DIVIDE (list 'TIMES 2 3) 0)) 'NaN)


ACL2S !>>(DEFSNAPSHOT TODO-8)

Summary
Form:  ( DEFLABEL TODO-8 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-8
ACL2S !>>(DEFINEC
          FOLD-CONSTANTS (TREE TREE)
          TREE
          (IF
           (RATIONALP TREE)
           TREE
           (IF
            (SYMBOLP TREE)
            TREE
            (IF
             (OR (EQUAL (FOLD-CONSTANTS (SECOND TREE))
                        'NAN)
                 (AND (EQUAL (FOLD-CONSTANTS (THIRD TREE))
                             'NAN)
                      (NOT (EQUAL (FIRST TREE) 'UMINUS))))
             'NAN
             (IF
              (AND (EQUAL (FIRST TREE) 'UMINUS)
                   (NOT (RATIONALP (FOLD-CONSTANTS (SECOND TREE)))))
              (LIST 'UMINUS (SECOND TREE))
              (IF
               (EQUAL (FIRST TREE) 'UMINUS)
               (- 0 (FOLD-CONSTANTS (SECOND TREE)))
               (IF
                (OR (NOT (RATIONALP (FOLD-CONSTANTS (SECOND TREE))))
              

## TODO-9 (10 points)

Let's make sure the program in TODO-8 works correctly. Use `test?` to verify the property that the value of an expression after folding constant terms does not change. E.g., the value of 

    (PLUS (TIMES 2 3) x)

is the same as the value of 

    (PLUS 6 x)

in any environment.

In [86]:
(defsnapshot todo-9)

(test? (implies (and (treep m) (envp e)) (equal (expr-eval-nan m e) (expr-eval-nan (fold-constants m) e))))


ACL2S !>>(DEFSNAPSHOT TODO-9)

Summary
Form:  ( DEFLABEL TODO-9 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-9
ACL2S !>>(TEST? (IMPLIES (AND (TREEP M) (ENVP E))
                         (EQUAL (EXPR-EVAL-NAN M E)
                                (EXPR-EVAL-NAN (FOLD-CONSTANTS M) E))))

**Summary of Cgen/testing**
We tested 3000 examples across 2 subgoals, of which 2944 (2944 unique)
satisfied the hypotheses, and found 0 counterexamples and 2944 witnesses.

Cases in which the conjecture is true include:
 [found in : "top"]
 -- ((E '((AABAA -1) (A -1) (BA 0))) (M '(DIVIDE (PLUS 0 0) 0)))
 -- ((E '((A 0) (A 0) (A 0))) (M '(UMINUS 0)))
 -- ((E NIL) (M '(MINUS 0 0)))

Test? succeeded. No counterexamples were found.

## TODO-10 (10 points)

Again, the automated testing should show no counterexamples, but the tests may not be terribly convincing, since it's not terribly likely that the random tree actually has something to optimize. So use `thm` to **prove** that `fold-constants` always returns an expression that evaluates to the same value.

In [87]:
(defsnapshot todo-10)


(thm (implies (and (treep m) (envp e)) (equal (expr-eval-nan m e) (expr-eval-nan (fold-constants m) e))))



ACL2S !>>(DEFSNAPSHOT TODO-10)

Summary
Form:  ( DEFLABEL TODO-10 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 TODO-10
ACL2S !>>(THM (IMPLIES (AND (TREEP M) (ENVP E))
                       (EQUAL (EXPR-EVAL-NAN M E)
                              (EXPR-EVAL-NAN (FOLD-CONSTANTS M) E))))

ACL2 Warning [Invariant-risk] in ACL2::TEST-CHECKPOINT:  Invariant-
risk has been detected for a call of function ACL2::TEST-CHECKPOINT
(as possibly leading to an ill-guarded call of CGEN::UI); see :DOC
invariant-risk.


ACL2 Warning [Invariant-risk] in ACL2::TEST-CHECKPOINT:  Invariant-
risk has been detected for a call of function ACL2::TEST-CHECKPOINT
(as possibly leading to an ill-guarded call of CGEN::UI); see :DOC
invariant-risk.


Name the formula above *1.

Perhaps we can prove *1 by induction.  Six induction schemes are suggested
by this conjecture.  Subsumption reduces that number to four.  These
merge into two derived induction schemes.  However, one of these i

## On Optimizing Compilers

We have two different optimizations now, so you can think of building a full-fledged optimizer by running `fold-constants` and then `remove-plus-0`. Or should it be first `remove-plus-0` and then `fold-constants`?

As things stand, `fold-constants` may produce a 0 that can then be removed by `remove-plus-0`, but there's no way for `remove-plus-0` to start with a non-constant expression and return a constant expression that `fold-constants` can work with. In other words, folding constants first, then removing zeros is the best strategy.

But let's think about adding some more optimizations, for example `(MINUS x x)` is the same as 0, right? This optimization can create a 0 that can then be removed by `remove-plus-0` or folded into another constant by `fold-constants`. Then again, we may need to run `remove-plus-0` to realize that two arguments to `MINUS` are the same! So in fact, there is no optimal ordering here, and a good optimizing compiler will keep trying the different optimizations until the expression is as optimized as it can get.

I won't ask you to build the global optimizer that keeps trying all the little optimizations until it's done, but I will give you the opportunity to try out some more local optimizations for extra credit. 

## Extra Credit 2 (Up to 90 points)

Implement some other optimizations. Here are some ideas to get you started:

* `(MINUS x x)`
* `(TIMES x 1)`
* `(TIMES x 0)`
* `(DIVIDE x 1)`
* `(UMINUS (UMINUS x))`
* ...

For each one, write the function that performs the optimization, use `test?` to make sure the optimization is correct, and use `thm` to prove that the optimization is correct. 30 points per optimization (up to 90 points, total).

In [88]:
(defsnapshot ec-2)
(definec remove-minus-double (tree :tree) :tree
    (if (or (rationalp tree) (symbolp tree))
        tree
        (if (or (equal (second tree) 'NaN) (equal (third tree) 'NaN))
            'NaN
            (if (equal (first tree) 'UMINUS)
                (cons (first tree) (list (remove-minus-double (second tree))))
                (if (equal (first tree) 'MINUS)
                    (if (and (and (or (symbolp (second tree)) (rationalp (second tree))) (or (symbolp (third tree)) (rationalp (third tree)))) (equal (second tree) (third tree)))
                        0
                        (append (cons (first tree) (list (remove-minus-double (second tree)))) (list (remove-minus-double (third tree))))
                    ) 
                    (append (cons (first tree) (list (remove-minus-double (second tree)))) (list (remove-minus-double (third tree))))
                )     
            ) 
        )
    )
)
(check-expect (remove-minus-double (list 'PLUS (list 'MINUS 'x 'x) 4)) (list 'PLUS 0 4))
(check-expect (remove-minus-double (list 'PLUS (list 'MINUS 9 9) 14)) (list 'PLUS 0 14))
(check-expect (remove-minus-double (list 'DIVIDE (list 'MINUS 'y 'y) 4)) (list 'DIVIDE 0 4))
(check-expect (remove-minus-double (list 'PLUS (list 'MINUS '6 'x) 4)) (list 'PLUS (list 'MINUS '6 'x) 4))
(check-expect (remove-minus-double (list 'PLUS (list 'MINUS 'x 'x) (list 'DIVIDE 15 (list 'MINUS (list 'MINUS 'y 'y) 5)))) (list 'PLUS 0 (list 'DIVIDE 15 (list 'MINUS 0 5))))

(test? (implies (and (treep m) (envp e)) (equal (expr-eval-nan m e) (expr-eval-nan (remove-minus-double m) e))))
(thm (implies (and (treep m) (envp e))  (equal (expr-eval-nan m e) (expr-eval-nan (remove-minus-double m) e))))

;/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

(definec remove-times-1 (tree :tree) :tree
    (if (or (rationalp tree) (symbolp tree))
        tree
        (if (equal (first tree) 'UMINUS)
            (cons (first tree) (list (remove-times-1 (second tree))))
            (if (equal (first tree) 'TIMES)
                (if (equal (second tree) 1)
                    (remove-times-1 (third tree))
                    (if (equal (third tree) 1)
                        (remove-times-1 (second tree))
                        (append (cons (first tree) (list (remove-times-1 (second tree)))) (list (remove-times-1 (third tree))))
                    )
                )  
                (append (cons (first tree) (list (remove-times-1 (second tree)))) (list (remove-times-1 (third tree))))
            )     
        ) 
    )
)
(check-expect (remove-times-1 (list 'MINUS (list 'TIMES 'x 1) 7)) (list 'MINUS 'x 7))
(check-expect (remove-times-1 (list 'MINUS (list 'TIMES 1 'x) 7)) (list 'MINUS 'x 7))
(check-expect (remove-times-1 (list 'TIMES (list 'DIVIDE 'x 1) 1)) (list 'DIVIDE 'x 1))
(check-expect (remove-times-1 (list 'MINUS (list 'TIMES 'x 1) (list 'TIMES 1 'y))) (list 'MINUS 'x 'y))
(check-expect (remove-times-1 (list 'TIMES (list 'TIMES 'z 1) 7)) (list 'TIMES 'z 7))

(test? (implies (and (treep m) (envp e)) (equal (expr-eval-nan m e) (expr-eval-nan (remove-times-1 m) e))))
(thm (implies (and (treep m) (envp e))  (equal (expr-eval-nan m e) (expr-eval-nan (remove-times-1 m) e))))

;/////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

(definec remove-divide-1 (tree :tree) :tree
    (if (or (rationalp tree) (symbolp tree))
        tree
        (if (equal (first tree) 'UMINUS)
            (cons (first tree) (list (remove-divide-1 (second tree))))
            (if (equal (first tree) 'DIVIDE)
                (if (equal (third tree) 1)
                    (remove-divide-1 (second tree))
                    (append (cons (first tree) (list (remove-divide-1 (second tree)))) (list (remove-divide-1 (third tree))))
                )   
                (append (cons (first tree) (list (remove-divide-1 (second tree)))) (list (remove-divide-1 (third tree))))
            )     
        ) 
    )
)
(check-expect (remove-divide-1 (list 'DIVIDE (list 'TIMES 'x 1) 1)) (list 'TIMES 'x 1))
(check-expect (remove-divide-1 (list 'MINUS (list 'DIVIDE 1 'x) 7)) (list 'MINUS (list 'DIVIDE 1 'x) 7))
(check-expect (remove-divide-1 (list 'TIMES (list 'DIVIDE 'x 1) 1)) (list 'TIMES 'x 1))
(check-expect (remove-divide-1 (list 'MINUS (list 'DIVIDE 'x 1) (list 'DIVIDE 'y 1))) (list 'MINUS 'x 'y))
(check-expect (remove-divide-1 (list 'TIMES (list 'DIVIDE 'z 1) 7)) (list 'TIMES 'z 7))

(test? (implies (and (treep m) (envp e)) (equal (expr-eval-nan m e) (expr-eval-nan (remove-divide-1 m) e))))
(thm (implies (and (treep m) (envp e))  (equal (expr-eval-nan m e) (expr-eval-nan (remove-divide-1 m) e))))


ACL2S !>>(DEFSNAPSHOT EC-2)

Summary
Form:  ( DEFLABEL EC-2 ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 EC-2
ACL2S !>>(DEFINEC
          REMOVE-MINUS-DOUBLE (TREE TREE)
          TREE
          (IF
           (OR (RATIONALP TREE) (SYMBOLP TREE))
           TREE
           (IF
            (OR (EQUAL (SECOND TREE) 'NAN)
                (EQUAL (THIRD TREE) 'NAN))
            'NAN
            (IF
             (EQUAL (FIRST TREE) 'UMINUS)
             (CONS (FIRST TREE)
                   (LIST (REMOVE-MINUS-DOUBLE (SECOND TREE))))
             (IF
                (EQUAL (FIRST TREE) 'MINUS)
                (IF (AND (AND (OR (SYMBOLP (SECOND TREE))
                                  (RATIONALP (SECOND TREE)))
                              (OR (SYMBOLP (THIRD TREE))
                                  (RATIONALP (THIRD TREE))))
                         (EQUAL (SECOND TREE) (THIRD TREE)))
                    0
                    (APPEND (CONS (FIRST TREE)
      